# Working with final df to scale everything.

In [52]:
import pandas as pd
from path import Path

import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd
import numpy as np
import timeit
import time
import datetime
import copy

import numpy as np
from sklearn import preprocessing
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [33]:
file_path = Path("../../../../data/cleandata/success_df.csv")

In [34]:
success_df = pd.read_csv(file_path)

In [35]:
success_df.head(3)

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,custom_index,custom_index_scaled,success
0,2,Ethereum,ETH,"20,155,411,771",185.67,"7,906,136,399",-0.10,-1.20,-0.96,Switzerland,...,0 days 00:00:00.000000000,0.460570,0.06116,0.02739,0.90151,100,779,144636.93,0.129162,1
1,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,0 days 00:00:00.000000000,0.563773,0.09667,0.04141,0.86189,100,848,17867.36,0.015956,1
2,8,EOS,EOS,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,Cayman Islands,...,0 days 00:00:00.000000000,0.828470,0.03220,0.85855,0.10927,100,522,1785.24,0.001594,0


# drop unncessary columns and scale all

In [36]:
success_df.columns

Index(['rank_cnc', 'Name', 'Ticker', 'market_cap', 'price', 'volume24',
       'pct1h', 'pct24h', 'pct7d', 'country', 'platform', 'pre_ico_end',
       'pre_ico_start', 'rating', 'status', 'type', 'End', 'Start', 'Duration',
       'pre_Duration', 'compound', 'negative', 'neutral', 'positive',
       'no_of_posts', 'Longevity', 'custom_index', 'custom_index_scaled',
       'success'],
      dtype='object')

In [37]:
success_df.drop(columns = ['Name', 'Ticker','country', 'platform', 'pre_ico_end',
       'pre_ico_start', 'status', 'type', 'pre_Duration','custom_index',
       'custom_index_scaled', 'success', 'End', 'Start'], inplace = True)

In [43]:
success_df['Duration'][1][:-23]

'20 '

In [49]:
def removing_days(df):
    
    for i in range(len(df)) : 
        df.loc[i, "Duration"] = df.loc[i, "Duration"][:-23]
    return df

In [50]:
removing_days(success_df)

,rank_cnc,market_cap,price,volume24,pct1h,pct24h,pct7d,rating,Duration,compound,negative,neutral,positive,no_of_posts,Longevity
0,2,"20,155,411,771",185.67,"7,906,136,399",-0.10,-1.20,-0.96,0.0,0,0.460570,0.061160,0.027390,0.901510,100,779
1,7,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,0.0,20,0.563773,0.096670,0.041410,0.861890,100,848
2,8,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,4.6,350,0.828470,0.032200,0.858550,0.109270,100,522
3,13,"1,105,888,653",0.042654,"53,437,224",-0.50,-1.72,-2.12,0.0,488,0.589474,0.098387,0.032946,0.868753,93,1049
4,16,"913,746,412",3.79,"252,881,495",-0.27,-1.89,-4.79,2.8,0,0.525717,0.081630,0.033480,0.884940,100,779
5,18,"821,746,518",1.24,"44,390,717",-0.25,10.13,2.47,4.3,13,0.663939,0.021939,0.878857,0.099327,49,855
6,19,"760,265,147",3.99,"147,439,544",-1.62,1.29,2.14,4.3,0,0.340496,0.060687,0.023224,0.916194,67,954
7,22,"632,251,075",69.07,"346,367,843",-0.12,-1.18,-5.06,0.0,0,0.663146,0.102770,0.025100,0.872110,100,779
8,23,"547,458,892",4.76,"642,121,534",-0.20,-2.10,-9.16,1.1,0,0.377329,0.079930,0.037850,0.882180,100,779
9,29,"347,237,523",0.256002,"73,875,875",-1.56,-1.64,1.16,3.5,0,0.830295,0.031600,0.837910,0.130530,100,899


In [53]:
success_scaled = StandardScaler().fit_transform(success_df)

ValueError: could not convert string to float: '20,155,411,771'